In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.models as models
import copy
from Images import Normalization

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cnn = models.vgg19(pretrained=True).to(device).features.eval()
print("Model's state_dict:")
for param_tensor in cnn.state_dict():
    print(param_tensor, "\t", cnn.state_dict()[param_tensor].size())

for layer in cnn.children():
    print (layer)

Model's state_dict:
0.weight 	 torch.Size([64, 3, 3, 3])
0.bias 	 torch.Size([64])
2.weight 	 torch.Size([64, 64, 3, 3])
2.bias 	 torch.Size([64])
5.weight 	 torch.Size([128, 64, 3, 3])
5.bias 	 torch.Size([128])
7.weight 	 torch.Size([128, 128, 3, 3])
7.bias 	 torch.Size([128])
10.weight 	 torch.Size([256, 128, 3, 3])
10.bias 	 torch.Size([256])
12.weight 	 torch.Size([256, 256, 3, 3])
12.bias 	 torch.Size([256])
14.weight 	 torch.Size([256, 256, 3, 3])
14.bias 	 torch.Size([256])
16.weight 	 torch.Size([256, 256, 3, 3])
16.bias 	 torch.Size([256])
19.weight 	 torch.Size([512, 256, 3, 3])
19.bias 	 torch.Size([512])
21.weight 	 torch.Size([512, 512, 3, 3])
21.bias 	 torch.Size([512])
23.weight 	 torch.Size([512, 512, 3, 3])
23.bias 	 torch.Size([512])
25.weight 	 torch.Size([512, 512, 3, 3])
25.bias 	 torch.Size([512])
28.weight 	 torch.Size([512, 512, 3, 3])
28.bias 	 torch.Size([512])
30.weight 	 torch.Size([512, 512, 3, 3])
30.bias 	 torch.Size([512])
32.weight 	 torch.Size([512, 5

In [24]:
PATH = 'test/vgg16.pth'
torch.save(model.state_dict(), PATH)

In [25]:
def gram_matrix(input):
        batch_size , h, w, f_map_num = input.size()
        features = input.view(batch_size * h, w * f_map_num)
        G = torch.mm(features, features.t())
        return G.div(batch_size * h * w * f_map_num)

class ContentLoss(nn.Module):

        def __init__(self, target,):
            super(ContentLoss, self).__init__()
            self.target = target.detach()
            self.loss = F.mse_loss(self.target, self.target )

        def forward(self, input):
            self.loss = F.mse_loss(input, self.target)
            return input


class StyleLoss(nn.Module):
        def __init__(self, target_feature):
            super(StyleLoss, self).__init__()
            self.target = gram_matrix(target_feature).detach()
            self.loss = F.mse_loss(self.target, self.target)# to initialize with something

        def forward(self, input):
            G = gram_matrix(input)
            self.loss = F.mse_loss(G, self.target)
            return input

In [26]:
normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# normalization = Normalization(normalization_mean, normalization_std).to(device)
model = nn.Sequential()

i = 0  # increment every time we see a conv
for layer in cnn.children():
    if isinstance(layer, nn.Conv2d):
        i += 1
        name = 'conv_{}'.format(i)
    elif isinstance(layer, nn.ReLU):
        name = 'relu_{}'.format(i)
        # The in-place version doesn't play very nicely with the ContentLoss
        # and StyleLoss we insert below. So we replace with out-of-place
        # ones here.
        #Переопределим relu уровень
        layer = nn.ReLU(inplace=False)
    elif isinstance(layer, nn.MaxPool2d):
        name = 'pool_{}'.format(i)
    elif isinstance(layer, nn.BatchNorm2d):
        name = 'bn_{}'.format(i)
    else:
        raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))
    model.add_module(name, layer)

In [27]:
new_model = nn.Sequential()
for name, module in model.named_children():
    new_model.add_module(name, module)
    if name == 'conv_5':
        new_model.add_module(name, module)
        break

In [28]:
new_model

Sequential(
  (conv_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu_1): ReLU()
  (conv_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu_2): ReLU()
  (pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu_3): ReLU()
  (conv_4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu_4): ReLU()
  (pool_4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [30]:
for param_tensor in new_model.state_dict():
    print(param_tensor, "\t", new_model.state_dict()[param_tensor].size())

conv_1.weight 	 torch.Size([64, 3, 3, 3])
conv_1.bias 	 torch.Size([64])
conv_2.weight 	 torch.Size([64, 64, 3, 3])
conv_2.bias 	 torch.Size([64])
conv_3.weight 	 torch.Size([128, 64, 3, 3])
conv_3.bias 	 torch.Size([128])
conv_4.weight 	 torch.Size([128, 128, 3, 3])
conv_4.bias 	 torch.Size([128])
conv_5.weight 	 torch.Size([256, 128, 3, 3])
conv_5.bias 	 torch.Size([256])


In [31]:
torch.save(new_model, './test/entire_model.pth')